In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    import zipfile
    with zipfile.ZipFile('/content/drive/MyDrive/DL Project/YoloV5DataSet.zip', 'r') as zip_ref:
        zip_ref.extractall('./')
except: 
    print("Local Machine")

In [1]:

!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 15627, done.
remote: Counting objects: 100% (234/234), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 15627 (delta 116), reused 138 (delta 66), pack-reused 15393
Receiving objects: 100% (15627/15627), 14.65 MiB | 10.19 MiB/s, done.
Resolving deltas: 100% (10644/10644), done.
zsh:1: command not found: pip


In [ ]:
import yaml
from pathlib import Path
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from yolov5.utils.dataloaders import LoadImagesAndLabels
from yolov5.models.yolo import Model
from yolov5.utils.plots import plot_results
from yolov5.utils.general import set_logging

In [ ]:
versionYolo = 'm'
sizeOfBatch = 32
inputShape = (640, 640)
epochs = 100
NoAnchors = 3
dev = torch.device('CUDA' if torch.cuda.is_available() else 'CPU')
print("Using the {} device".format(dev))


In [ ]:
# Load configuration for dataset
with open('dataset.yaml') as f:
    data1 = yaml.safe_load(f)
trainPath, valPath = data1['train'], data1['val']
nc, names = data1['nc'], data1['names']

In [ ]:
#output directory
dirSave = Path('runs/train/signboard_exp')
(dirSave / 'weights').mkdir(parents=True, exist_ok=True)
set_logging(str(dirSave / 'train.log'))


In [ ]:
def collateFunc(batch):
    maxLen = max([t.shape[0] for _, t, _, _ in batch])
    imgs = torch.stack([img for img, _, _, _ in batch])
    targets = [torch.cat([t, torch.zeros(maxLen - t.shape[0], t.shape[1])], dim=0) for _, t, _, _ in batch]
    targets = torch.stack(targets)
    pathvals = [path for _, _, path, _ in batch]
    extras = [e for _, _, _, e in batch]

    return imgs, targets, pathvals, extras

In [ ]:
# Load the dataset
dsTrain = LoadImagesAndLabels(trainPath, 640, 16, rect=True, pad=0.5)
dsVal = LoadImagesAndLabels(valPath, 640, 16, rect=True, pad=0.5)
loaderTrain = DataLoader(dsTrain, batch_size=16, collate_fn=collateFunc,shuffle=True, num_workers=4, pin_memory=True)
loaderVal = DataLoader(dsVal, batch_size=16, collate_fn=collateFunc,shuffle=False, num_workers=4, pin_memory=True)


In [ ]:
print("Downloading Weights of yolo5 Verion ", versionYolo)
urlWeight = "https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5{}.pt".format(versionYolo)
!wget {urlWeight}

In [ ]:
def yoloCreation(classes, ver):
    fileConfig = "yolov5/models/yolov5{}.yaml".format(ver)
    model = Model(fileConfig, ch=3, nc=classes)
    ck = torch.load(f'yolov5{ver}.pt', map_location=dev)
    ck_model_dict = ck['model'].state_dict()
    compatible_weights = {k: v for k, v in ck_model_dict.items() if k in model.state_dict() and model.state_dict()[k].shape == v.shape}
    model.load_state_dict(compatible_weights, strict=False)
    model.hyp = ck['model'].hyp
    return model


In [ ]:

def TargeToTensor(targets, sizeOfBatch, anchorsNum, sizeOfGrids):
    objTarget = []
    boxTarget = []
    for sizeOfGrid in sizeOfGrids:
        objTarget.append(torch.zeros((sizeOfBatch, anchorsNum, sizeOfGrid, sizeOfGrid, 1)))
        boxTarget.append(torch.zeros((sizeOfBatch, anchorsNum, sizeOfGrid, sizeOfGrid, 4)))

    for indexBatch, tar in enumerate(targets):
        x1, y1, x2, y2 = tar.long()
        xcenter, ycenter, width, height = (x1 + x2) / 2, (y1 + y2) / 2, x2 - x1, y2 - y1

        for i, sizeOfGrid in enumerate(sizeOfGrids):
            xCell, yCell = int(xcenter * sizeOfGrid), int(ycenter * sizeOfGrid)
            anchor = 0
            try:
                objTarget[i][indexBatch, anchor, yCell, xCell, 0] = 1
                boxTarget[i][indexBatch, anchor, yCell, xCell] = torch.tensor([xcenter, ycenter, width, height])
            except Exception as e:
                pass
    return objTarget, boxTarget

In [ ]:

class LossOfSignboard(nn.Module):
    def __init__(self, NoAnchors=3):
        super(LossOfSignboard, self).__init__()
        self.NoAnchors = NoAnchors

    def forward(self, preds, targets):
        lossObj = torch.tensor(0.0, device=preds[0].device)
        lossBox = torch.tensor(0.0, device=preds[0].device)
        sizeOfBatch = preds[0].size(0)
        sizeOfGrids = [pred.size(2) for pred in preds]
        targetObjList, targeBoxList = TargeToTensor(targets, sizeOfBatch, self.NoAnchors, sizeOfGrids)

        for i, pred in enumerate(preds):
            objTarget = targetObjList[i].to(pred.device)
            boxTarget = targeBoxList[i].to(pred.device)

            lossObj += nn.BCEWithLogitsLoss()(pred[..., 4:5], objTarget)
            lossBox += nn.MSELoss()(pred[..., :4], boxTarget)

        totalLoss = lossObj + lossBox
        return totalLoss

In [ ]:
# Load model
classes = 47
model = yoloCreation(classes,versionYolo)
model = model.to(dev)
opt = optim.SGD(model.parameters(), lr=0.01, momentum=0.937, weight_decay=0.0005, nesterov=True)
criteria = LossOfSignboard()
criteria = criteria.to(dev)


In [ ]:

epochs = 100
bestFit = float('inf')

for epoch in range(epochs):
    model.train()

    for i, (imgs, targets, paths, _) in enumerate(loaderTrain):
        imgs = imgs.to(dev).float() / 255.0
        targets = targets.to(dev)
        pred = model(imgs)
        print("Pred shape:")
        for p in pred:
            print(p.shape)
        print("Target shape:", targets.shape)
        loss, lossItem = criteria(pred, targets)
        lossItem = torch.cat(lossItem)
        loss.backward()
        opt.step()
        opt.zero_grad()
    model.eval()

    with torch.no_grad():
        for i, (imgs, targets, paths, _) in enumerate(loaderVal):
            imgs = imgs.to(dev).float() / 255.0
            targets = targets.to(dev)

            pred = model(imgs)
            print("Pred shape:", pred.shape)
            print("Targets shape:", targets.shape)
            lossVal, lossValItem = criteria(pred, targets)
            lossValItem = torch.cat(lossValItem)

    # Save best model
    if lossVal < bestFit:
        bestFit = lossVal
        torch.save(model.state_dict(), dirSave / 'weights' / 'best.pt')

# Plot training results
plot_results(dirSave=dirSave)
